In [1]:
import pandas as pd
import numpy as np

In [32]:
CV = 5
TOP_COLS = 30 - 1
TOP_GRPS = 4

## HMPv13_otu_table

In [ ]:
# Project ID : 1928
# Subject ID: SRS 
# Experiment ID: SRX
# Run ID: SRR

# df = pd.read_csv('/projects/genomic-ml/da2343/ml_project_1/data/microbe_ds/HMPv13_otu_table.csv').T
df = pd.read_csv('/projects/genomic-ml/da2343/ml_project_1/data/microbe_ds/HMPv35_otu_table.csv').T
# Convert the index to a series and split it
id_df = df.index.to_series().str.split('.', expand=True)
# Rename the columns as per your requirement
id_df.columns = ['Project_ID', 'SRS_ID', 'SRX_ID', 'SRR_ID']
# Now 'id_df' has the separate columns. You can join this back to your original DataFrame if needed
# join infront of the df
df = df.join(id_df).reset_index(drop=True)
df = df.sort_values('SRX_ID')
# drop the columns that are not needed
df = df.drop(['Project_ID', 'SRS_ID', 'SRR_ID'], axis=1)
# rename the SRX_ID column to Sample_ID
df = df.rename(columns={'SRX_ID': 'Group_ID'}).reset_index(drop=True)

# give the group ID a number each
df['Group_ID'] = (pd.factorize(df['Group_ID'])[0] + 1)

# drop the Group_ID column
# df = df.drop(['Group_ID'], axis=1)
# df = np.log(df + 1)
# df.to_csv('/projects/genomic-ml/da2343/ml_project_1/data/microbe_ds/HMPv13_otu_table_log.csv')

df['Group_ID'] = df['Group_ID'].astype(int)
df

# df_selected is without the Group_ID column
df_selected = df.drop(['Group_ID'], axis=1) 
df_selected = np.log10(df_selected + 1)
df_selected['Group_ID'] = df['Group_ID']
df_selected

# First, find the minimum group size
min_group_size = int(df_selected['Group_ID'].value_counts().mean())

# but min_group_size should be a multiple of the cv constant which is 5
min_group_size = min_group_size - (min_group_size % CV)

# remove all groups that are smaller than the min_group_size
df_selected = df_selected.groupby('Group_ID').filter(lambda x: len(x) >= min_group_size)

# Now, use groupby and sample to get equal-sized samples from each group
equal_samples_df = df_selected.groupby('Group_ID').apply(lambda x: x.sample(min_group_size)).reset_index(drop=True)

# arrange Group_ID in ascending order
equal_samples_df = equal_samples_df.sort_values('Group_ID').reset_index(drop=True)
equal_samples_df

In [36]:
# equal_samples_df.to_csv('/projects/genomic-ml/da2343/ml_project_1/data/microbe_ds/HMPv13_sub_log_update.csv', index=False)
equal_samples_df.to_csv('/projects/genomic-ml/da2343/ml_project_1/data/microbe_ds/HMPv35_sub_log_update.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np

# Read and transform the data
df = pd.read_csv('/projects/genomic-ml/da2343/ml_project_1/data/microbe_ds/HMPv35_otu_table.csv').T

# Process IDs
id_df = df.index.to_series().str.split('.', expand=True)
id_df.columns = ['Project_ID', 'SRS_ID', 'SRX_ID', 'SRR_ID']
df = df.join(id_df).reset_index(drop=True)
df = df.sort_values('SRX_ID')
df = df.drop(['Project_ID', 'SRS_ID', 'SRR_ID'], axis=1)
df = df.rename(columns={'SRX_ID': 'Group_ID'}).reset_index(drop=True)
df['Group_ID'] = (pd.factorize(df['Group_ID'])[0] + 1)

# Convert Group_ID to int and filter by TOP_GRPS
df['Group_ID'] = df['Group_ID'].astype(int)


def filter_informative_columns(df, zero_threshold=0.6, n_top_columns=30):
    """
    Filter columns based on:
    1. Percentage of zero entries (remove if too many zeros)
    2. Sort by non-zero entries and take top columns
    
    Parameters:
    - df: DataFrame
    - zero_threshold: maximum allowed fraction of zeros (default 0.8)
    - n_top_columns: number of top columns to select (default 20)
    
    Returns:
    - DataFrame with filtered columns
    """
    # Separate Group_ID
    group_id = df['Group_ID']
    data_cols = df.drop('Group_ID', axis=1)
    
    # Calculate fraction of zeros in each column
    zero_fractions = (data_cols == 0).mean()
    
    # Get columns that pass the zero threshold
    mask = (zero_fractions < zero_threshold)
    
    # Sort columns by non-zero fraction and take top n
    scores = (1 - zero_fractions[mask])
    top_cols = scores.sort_values(ascending=False).head(n_top_columns).index
    
    # Create final DataFrame
    df_filtered = data_cols[top_cols]
    df_filtered = np.log10(df_filtered + 1)
    df_filtered['Group_ID'] = group_id
    
    # Print some information about the filtering
    print(f"Original number of columns: {len(data_cols.columns)}")
    print(f"Columns after zero threshold: {len(scores)}")
    print(f"Final number of columns: {len(top_cols)}")
    
    return df_filtered

# Apply the filtering
df_selected = filter_informative_columns(df)

# First, find the minimum group size
min_group_size = int(df_selected['Group_ID'].value_counts().mean())

# but min_group_size should be a multiple of the cv constant which is 5
min_group_size = min_group_size - (min_group_size % CV)

# remove all groups that are smaller than the min_group_size
df_selected = df_selected.groupby('Group_ID').filter(lambda x: len(x) >= min_group_size)

# Now, use groupby and sample to get equal-sized samples from each group
equal_samples_df = df_selected.groupby('Group_ID').apply(lambda x: x.sample(min_group_size)).reset_index(drop=True)

# arrange Group_ID in ascending order
equal_samples_df = equal_samples_df.sort_values('Group_ID').reset_index(drop=True)

# Calculate group sparsity and select top groups based on least sparse groups
group_sparsity = equal_samples_df.groupby('Group_ID').apply(
    lambda x: (x.drop('Group_ID', axis=1) == 0).mean().mean()
).sort_values()  # Sort groups by their sparsity

# Take the top N least sparse groups
top_groups = group_sparsity.head(TOP_GRPS).index
equal_samples_df = equal_samples_df[equal_samples_df['Group_ID'].isin(top_groups)]
equal_samples_df

In [37]:
equal_samples_df.to_csv('/projects/genomic-ml/da2343/ml_project_1/data/microbe_ds/HMPv35_filtered.csv', index=False)

In [ ]:
equal_samples_df

## MovingPictures_otu_table

In [ ]:
df = pd.read_csv('/projects/genomic-ml/da2343/ml_project_1/data/microbe_ds/MovingPictures_otu_table.csv').T
id_df = df.index.to_series().str.split('.', expand=True)
id_df.columns = ['Project_ID', 'Sample_ID', 'S_Constant', 'Group_ID', 'Sequence_Keyword']
df = df.join(id_df).reset_index(drop=True)
df = df.sort_values('Group_ID')
df = df.drop(['Project_ID', 'Sample_ID', 'S_Constant', 'Sequence_Keyword'], axis=1).reset_index(drop=True)

# drop the Group_ID column
# df = df.drop(['Group_ID'], axis=1)
# df = np.log(df + 1)
# df.to_csv('/projects/genomic-ml/da2343/ml_project_1/data/microbe_ds/MovingPictures_otu_table_log.csv')
# df

df['Group_ID'] = (pd.factorize(df['Group_ID'])[0] + 1)
df['Group_ID'] = df['Group_ID'].astype(int)

# df_selected is without the Group_ID column
df_selected = df.drop(['Group_ID'], axis=1) 
df_selected = np.log10(df_selected + 1)
df_selected['Group_ID'] = df['Group_ID']
df_selected

# column_sums = df.sum(axis=0).sort_values(ascending=False)
# top_20_columns = column_sums.head(TOP_COLS).index
# df_selected = df[top_20_columns]
# df_selected = np.log10(df_selected + 1)
# df_selected['Group_ID'] = df['Group_ID']

# First, find the minimum group size
min_group_size = int(df_selected['Group_ID'].value_counts().mean())

# but min_group_size should be a multiple of the cv constant which is 5
min_group_size = min_group_size - (min_group_size % CV)

# remove all groups that are smaller than the min_group_size
df_selected = df_selected.groupby('Group_ID').filter(lambda x: len(x) >= min_group_size)

# Now, use groupby and sample to get equal-sized samples from each group
equal_samples_df = df_selected.groupby('Group_ID').apply(lambda x: x.sample(min_group_size)).reset_index(drop=True)

# arrange Group_ID in ascending order
equal_samples_df = equal_samples_df.sort_values('Group_ID').reset_index(drop=True)
equal_samples_df

In [38]:
equal_samples_df.to_csv('/projects/genomic-ml/da2343/ml_project_1/data/microbe_ds/MovingPictures_sub_log_update.csv', index=False)

## qa10394_otu_table

In [ ]:
df = pd.read_csv('/projects/genomic-ml/da2343/ml_project_1/data/microbe_ds/qa10394_otu_table.csv').T
id_df = df.index.to_series().str.split('.', expand=True)
# use only column with index 1
id_df = id_df.iloc[:, 1]
id_df = id_df.rename('Group_ID')
df = df.join(id_df).reset_index(drop=True)
df = df.sort_values('Group_ID')
df = df[(df.Group_ID != "BLANK") & (df.Group_ID != "mistake")]
df = df.reset_index(drop=True)


# drop the Group_ID column
#df = df.drop(['Group_ID'], axis=1)
#df = np.log(df + 1)
#df.to_csv('/projects/genomic-ml/da2343/ml_project_1/data/microbe_ds/qa10394_otu_table_log.csv')

df['Group_ID'] = (pd.factorize(df['Group_ID'])[0] + 1)
df['Group_ID'] = df['Group_ID'].astype(int)

# df_selected is without the Group_ID column
df_selected = df.drop(['Group_ID'], axis=1) 
df_selected = np.log10(df_selected + 1)
df_selected['Group_ID'] = df['Group_ID']
df_selected

# First, find the minimum group size
min_group_size = int(df_selected['Group_ID'].value_counts().mean())

# but min_group_size should be a multiple of the cv constant which is 5
min_group_size = min_group_size - (min_group_size % CV)

# remove all groups that are smaller than the min_group_size
df_selected = df_selected.groupby('Group_ID').filter(lambda x: len(x) >= min_group_size)

# Now, use groupby and sample to get equal-sized samples from each group
equal_samples_df = df_selected.groupby('Group_ID').apply(lambda x: x.sample(min_group_size)).reset_index(drop=True)

# arrange Group_ID in ascending order
equal_samples_df = equal_samples_df.sort_values('Group_ID').reset_index(drop=True)
equal_samples_df

In [41]:
equal_samples_df.to_csv('/projects/genomic-ml/da2343/ml_project_1/data/microbe_ds/qa10394_sub_log_update.csv', index=False)

## TwinsUK_otu_table

In [ ]:
df = pd.read_csv('/projects/genomic-ml/da2343/ml_project_1/data/microbe_ds/TwinsUK_otu_table.csv').T
id_df = df.index.to_series().str.split('.', expand=True)
id_df = id_df.iloc[:, 1]
id_df = id_df.rename('Group_ID')
df = df.join(id_df).reset_index(drop=True)
df = df.sort_values('Group_ID').reset_index(drop=True)


# drop the Group_ID column
#df = df.drop(['Group_ID'], axis=1)
#df = np.log(df + 1)
#df.to_csv('/projects/genomic-ml/da2343/ml_project_1/data/microbe_ds/TwinsUK_otu_table_log.csv')
df['Group_ID'] = (pd.factorize(df['Group_ID'])[0] + 1)
df['Group_ID'] = df['Group_ID'].astype(int)

# df_selected is without the Group_ID column
df_selected = df.drop(['Group_ID'], axis=1) 
df_selected = np.log10(df_selected + 1)
df_selected['Group_ID'] = df['Group_ID']
# df_selected

# First, find the minimum group size
min_group_size = int(df_selected['Group_ID'].value_counts().mean())

# but min_group_size should be a multiple of the cv constant which is 5
min_group_size = min_group_size - (min_group_size % CV)

# remove all groups that are smaller than the min_group_size
df_selected = df_selected.groupby('Group_ID').filter(lambda x: len(x) >= min_group_size)

# Now, use groupby and sample to get equal-sized samples from each group
equal_samples_df = df_selected.groupby('Group_ID').apply(lambda x: x.sample(min_group_size)).reset_index(drop=True)

# arrange Group_ID in ascending order
equal_samples_df = equal_samples_df.sort_values('Group_ID').reset_index(drop=True)
equal_samples_df

In [43]:
equal_samples_df.to_csv('/projects/genomic-ml/da2343/ml_project_1/data/microbe_ds/TwinsUK_sub_log_update.csv', index=False)